In [1]:
import pickle
import numpy as np
import os

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

import open3d as o3d

from collections import defaultdict

In [2]:
%matplotlib inline

In [3]:
%matplotlib qt

In [4]:
output_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/"

# Save input pointcloud

In [ ]:
obj_names = ["005_tomato_soup_can", "035_power_drill", "052_extra_large_clamp", "065-i_cups"]

sel_grasps = {
    "5": ["3"],
    "10": ["3"],
    "50": ["0"]
}

input_pointcloud_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/ycb_different_grasps_2/ycb_con_grasps_2/"

In [82]:
output_pointcloud_dir = os.path.join(output_dir, "input_pointcloud")

for sel_grasp in sel_grasps:
    for grasp_num in sel_grasps[sel_grasp]:
        for obj_name in obj_names:
            input_pointcloud_pth = os.path.join(input_pointcloud_dir, sel_grasp + "_grasps", obj_name + "_" + sel_grasp + "-" + grasp_num, "pointcloud.npz")
            input_pointcloud = o3d.geometry.PointCloud()
            input_pointcloud.points = o3d.utility.Vector3dVector(np.load(input_pointcloud_pth)['points'])
            
            out_pth = os.path.join(output_pointcloud_dir, obj_name + "_" + sel_grasp + "-" + grasp_num + ".ply")
            o3d.io.write_point_cloud(out_pth, input_pointcloud)

# Reconstruct objects

In [5]:
ply_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/input_pointcloud/"
out_BallPivoting_dir = os.path.join(output_dir, "reconstruction_BallPivoting")

if not os.path.exists(out_BallPivoting_dir):
    os.makedirs(out_BallPivoting_dir)
    
out_Poisson_dir = os.path.join(output_dir, "reconstruction_Poisson")
if not os.path.exists(out_Poisson_dir):
    os.makedirs(out_Poisson_dir)

In [6]:
%%capture
for obj_name in os.listdir(ply_dir):
    ply_pth = os.path.join(ply_dir, obj_name)
    
    # Load pointcloud
    pcd_in = o3d.io.read_point_cloud(ply_pth)

    # Estimate normals and orient them
    pcd_in.estimate_normals()
    pcd_in.orient_normals_towards_camera_location(camera_location=np.array([0., 0., 0.]))
    normals = np.asarray(pcd_in.normals)
    pcd_in.normals = o3d.utility.Vector3dVector(- normals)

    # Reconstruct object Ball pivoting
    radii = [0.005, 0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14]

    rec_BP_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd_in, o3d.utility.DoubleVector(radii))
    rec_BP_mesh.paint_uniform_color([0.7, 0.7, 0.7])
    # o3d.visualization.draw_geometries([rec_mesh], mesh_show_back_face=True)

    # Save object Ball pivoting
    out_pth = os.path.join(out_BallPivoting_dir, obj_name)
    o3d.io.write_triangle_mesh(out_pth, rec_BP_mesh)
    
    # Reconstruction Poisson
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        rec_P_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_in, depth=9)
    rec_P_mesh.compute_triangle_normals()
    #rec_P_mesh.compute_vertex_normals()
    rec_P_mesh.paint_uniform_color([0.7, 0.7, 0.7])
    
    # Save object Poisson
    out_pth = os.path.join(out_Poisson_dir, obj_name)
    o3d.io.write_triangle_mesh(out_pth, rec_P_mesh)

In [22]:
for obj_name in os.listdir(ply_dir):
    print(obj_name)

005_tomato_soup_can_10-3.ply
005_tomato_soup_can_5-3.ply
005_tomato_soup_can_50-0.ply
035_power_drill_10-3.ply
035_power_drill_5-3.ply
035_power_drill_50-0.ply
052_extra_large_clamp_10-3.ply
052_extra_large_clamp_5-3.ply
052_extra_large_clamp_50-0.ply
065-i_cups_10-3.ply
065-i_cups_5-3.ply
065-i_cups_50-0.ply


# Change names

## No DL

In [2]:
import shutil

img_pth = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/img_noDL/"
img_out = "C:/Users/lucas/Desktop/UPC/TFM/images/simulation/noDL/"
translator = {
    "01": "05_pcd",
    "02": "10_pcd",
    "03": "50_pcd",
    "04": "05_BP",
    "05": "10_BP",
    "06": "50_BP",
    "07": "05_PD",
    "08": "10_PD",
    "09": "50_PD",
    "10": "GT"    
}

In [168]:

for img_name in os.listdir(img_pth):
    obj_name = img_name[:-6]
    obj_type = img_name[-6:-4]
    
    new_name = obj_name + "_" + translator[obj_type] + ".png"
    
    in_pth = os.path.join(img_pth, img_name)
    out_pth = os.path.join(img_out, new_name)
    shutil.copyfile(in_pth, out_pth)
    
    

## DL

In [1]:
import shutil
import os

img_pth = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/reconstruct_object/img_DL/"
img_out = "C:/Users/lucas/Desktop/UPC/TFM/images/simulation/DL/"
translator = {
    "11": "05_vanilla",
    "12": "10_vanilla",
    "13": "50_vanilla",
    "14": "05_sr",
    "15": "10_sr",
    "16": "50_sr",
    "17": "05_sr_grasps",
    "18": "10_sr_grasps",
    "19": "50_sr_grasps",
    "20": "05_grasps",
    "21": "10_grasps",
    "22": "50_grasps",
    "23": "05_pretrained_sn",
    "24": "10_pretrained_sn",
    "25": "50_pretrained_sn"
}

In [2]:

for img_name in os.listdir(img_pth):
    obj_name = img_name[:-6]
    obj_type = img_name[-6:-4]
    
    new_name = obj_name + "_" + translator[obj_type] + ".png"
    
    in_pth = os.path.join(img_pth, img_name)
    out_pth = os.path.join(img_out, new_name)
    shutil.copyfile(in_pth, out_pth)
    
    